# Import Data

In [114]:
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_absolute_error
from numpy import mean
from numpy import std

In [115]:
import warnings 
warnings.filterwarnings('ignore')

# Data PreProcess + Concatination

## Data Concatination

In [116]:
df = pd.concat(map(pd.read_csv,glob.glob('./datasets/*.csv')))
display(df)

,waktu,arah,koridor,target_kecepatan,jarak,target_waktu_tempuh,pencapaian_kecepatan_per_koridor,pencapaian_kecepatan,pencapaian_waktu_tempuh,pencapaian_Waktu_tempuh
0,pagi,Selatan - Pusat,Lenteng Agung - Ps. Minggu - Supomo - dr. Saha...,21.0,19.62,56.06,22.77,24.36,58.82,NaN
1,pagi,Selatan - Pusat,Cilandak KKO - Warung Buncit - Mampang Prapata...,21.0,13.36,38.17,24.14,24.36,36.88,NaN
2,pagi,Selatan - Pusat,Pangeran Antasari - Prapanca - Iskandarsyah - ...,21.0,11.13,31.80,29.64,24.36,25.99,NaN
3,pagi,Selatan - Pusat,Pondok Labu - Fatmawati - Panglima Polim - Sis...,21.0,16.25,46.43,25.82,24.36,44.35,NaN
4,pagi,Selatan - Pusat,Pasar Jumat - Metro Pondok Indah - Iskandar Mu...,21.0,15.87,45.34,28.18,24.36,37.33,NaN
...,...,...,...,...,...,...,...,...,...,...
77,sore,Pusat - Utara,Letjend. Suprapto - Yos Sudarso - Bollevard Ba...,21.0,20.29,57.97,22.03,23.02,56.04,NaN
78,sore,Pusat - Barat,Hayam Wuruk - Hasyim Ashari - Kyai Tapa - Daan...,21.0,14.47,41.34,20.79,21.11,45.71,NaN
79,sore,Pusat - Barat,Hayam Wuruk - Balikpapan - Tomang Raya - S. Pa...,21.0,15.55,44.43,17.91,21.11,55.19,NaN
80,sore,Pusat - Barat,Kebon Sirih - Jatibaru - Katamso - Kemanggisan...,21.0,17.43,49.80,23.41,21.11,52.63,NaN


## Drop Kolom pencapain_Waktu_tempuh

<p style="font-size:20px"><strong>91.684</strong> % dari kolom pencapaian_Waktu_tempuh adalah data kosong sehingga saya memutuskan untuk mendrop saja kolom ini</p>

In [117]:
dataRow = df.shape[0]
nullPencapainWaktuTempuh = df['pencapaian_Waktu_tempuh'].isnull().sum()
print(f'Dataframe memiliki {dataRow} baris')
print(f'Kolom pencapain_Waktu_tempuh memiliki {nullPencapainWaktuTempuh} baris yang NaN')
print(f'{round(((nullPencapainWaktuTempuh/dataRow)*100),3)} % dari kolom pencapain_Waktu_tempuh adalah data Nan')

Dataframe memiliki 986 baris
Kolom pencapain_Waktu_tempuh memiliki 904 baris yang NaN
91.684 % dari kolom pencapain_Waktu_tempuh adalah data Nan


In [118]:
df.drop(['pencapaian_Waktu_tempuh'],axis=1,inplace=True)

In [119]:
dataShape = df.shape
numOfNanRows = df.isnull().sum().sum()
print(f'Shape DataFrame sebelum data NaN dihapus: {dataShape}')
print(f'Jumlah baris yang NaN adalah sebanyak {numOfNanRows} baris')
# Drop nan rows in dataframe
text = ' AFTER DROPPING NAN VALUES '
print(f'\n{text:-^50}\n')
df.dropna(inplace = True)

dataShape = df.shape
numOfNanRows = df.isnull().sum().sum()
print(f'Shape DataFrame stelah data NaN dihapus: {dataShape}')
print(f'Jumlah baris yang NaN adalah sebanyak {numOfNanRows} baris')

Shape DataFrame sebelum data NaN dihapus: (986, 9)
Jumlah baris yang NaN adalah sebanyak 141 baris

----------- AFTER DROPPING NAN VALUES ------------

Shape DataFrame stelah data NaN dihapus: (902, 9)
Jumlah baris yang NaN adalah sebanyak 0 baris


# Nomor 1

In [120]:
numericDfIndex = [i for i in range(len(df.dtypes)) if df.dtypes[i] == 'float64']
numericDf = df.iloc[:,numericDfIndex]
display(numericDf)

,target_kecepatan,jarak,target_waktu_tempuh,pencapaian_kecepatan_per_koridor,pencapaian_kecepatan,pencapaian_waktu_tempuh
0,21.0,19.62,56.06,22.77,24.36,58.82
1,21.0,13.36,38.17,24.14,24.36,36.88
2,21.0,11.13,31.80,29.64,24.36,25.99
3,21.0,16.25,46.43,25.82,24.36,44.35
4,21.0,15.87,45.34,28.18,24.36,37.33
...,...,...,...,...,...,...
77,21.0,20.29,57.97,22.03,23.02,56.04
78,21.0,14.47,41.34,20.79,21.11,45.71
79,21.0,15.55,44.43,17.91,21.11,55.19
80,21.0,17.43,49.80,23.41,21.11,52.63


In [121]:
X = numericDf.drop(['pencapaian_kecepatan'],axis=1)
y = numericDf['pencapaian_kecepatan']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [122]:
numericDf.isnull().sum().sum()

0

In [123]:
model = LinearRegression()
folds = KFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_val_score(model, X, y, scoring='r2', cv=folds,n_jobs=-1)
meanScores = round(scores.mean()*100,3)
meanStd = round(scores.std()*100,3)
print(f'Linear Regression Mean Score = {meanScores}')
print(f'Linear Regression Standard Deviation = {meanStd}')

Linear Regression Mean Score = 64.166
Linear Regression Standard Deviation = 6.397


In [124]:
model = LinearRegression()
model.fit(X_train,y_train)
y_hat = model.predict(X_test)
mae = round(mean_absolute_error(y_test,y_hat),3)
print(f'Mean Absolute Error = {mae}')

Mean Absolute Error = 1.956


# Nomor 2